In [1]:
# empty list
data_list = []

# need to ensure a clean run each time
with open('day10_data.txt') as f:
    for line in f:
        data_list.append(line.strip('\n').split(' '))
       
       
asteroid_dict = {}


for i in range(len(data_list)):
   
    # convert to list of vals
    limit_row = list(data_list[i][0])
   
    
    for j in range(len(limit_row)):
        
        if limit_row[j] == '#':
           
            # add to asteroid dict
            asteroid_dict[(j,i)] = 'asteroid'

In [2]:
from scipy.spatial import distance


# new function: euclidean distance off coord - total laziness
def eucl_funct(coord1, coord2):
    
    return distance.euclidean(coord1, coord2)

In [3]:
def dir_x_funct(asteroid_x, val2):
    
    diff = val2 - asteroid_x
   
    if diff < 0:
       
        return "Negative"
   
    else:
        return "Positive"

def dir_y_funct(asteroid_y, val2):
    
    diff = val2 - asteroid_y # due to y_val axis being "backwards" we switch
   
    if diff < 0:
       
        return "Negative"
   
    else:
        return "Positive"
    
# equation to product slope
def slope_funct(coord1, coord2):
    
    # unpack
    x1,y1 = coord1
    x2,y2 = coord2
    
    try:
        return (y2 - y1) / (x2-x1) # slope is slightly diff.
   
    except:
        return None # need to be extreme here....sloppy sloppy
    
# x & y 
def int_funct(coord1, coord2):
    """ Calculate intercept"""
    
    # unpack
    x1,y1 = coord1
    x2,y2 = coord2
    
    if x1 == x2:
        return None # no y intercept
    
    slope = slope_funct(coord1, coord2)
    return -x1*slope+y1

# revuild y-val
def y_scale(y_asteroid, y_comp):
    """ Rescale from 0 - N to +N to -N with y_asteroid = 0"""
    
    return y_asteroid - y_comp

In [4]:
coord1 = (19,11)
y_zero = coord1[1]
new_y = y_scale(y_zero, coord1[1])

# dl from asteroid
try:
    del asteroid_dict[coord1]
except:
    print("Already removed")
    
coord1 = (coord1[0], new_y)

# build a few dfs to store info: 

total_vert = []
total_neg_vert = []
zone_1 = []
zone_2 = []
zone_3 = []
zone_4 = []

# count total distinct
for coord2 in asteroid_dict.keys():
     
    # update y_vals 
    new_y = y_scale(y_zero, coord2[1])
    coord2 = (coord2[0], new_y)
    
    # calculate slope & intercept:
    slope = slope_funct(coord1, coord2)
    intercept = int_funct(coord1, coord2)

    # DIRECTION OF X & Y RELATIVE TO ASTEROID
    dir_x = dir_x_funct(coord1[0], coord2[0])
    dir_y = dir_y_funct(coord1[1], coord2[1])
    
    # calculate euclidean distance
    eucl_dist = eucl_funct(coord1, coord2)
    
    # build tuple 
    coord2 = (slope, eucl_dist, coord2)

    # See if there is a starting value:
    if slope == None and dir_y == "Positive":
        total_vert.append(coord2)
        
    elif slope == None and dir_y == "Negative":
        total_neg_vert.append(coord2)        
    
    # add to list for zone 1
    elif slope >= 0 and dir_x == "Positive" and dir_y == "Positive":
        zone_1.append(coord2)
    
    # elif to list for zone 2
    elif slope <= 0 and dir_x == "Positive" and dir_y == "Negative":
        zone_2.append(coord2)

    # elif to list for zone 3
    elif slope >= 0 and dir_x == "Negative" and dir_y == "Negative":
        zone_3.append(coord2)

    # elif to list for zone 3
    elif slope <= 0 and dir_x == "Negative" and dir_y == "Positive":
        zone_4.append(coord2)
    else:
        print("Issue")

In [5]:
def blast_asteroids(slope, zone_list, rank_val):

    temp_vals = []

    for lines in zone_list:

        if lines[0] == slope:
            temp_vals.append(lines)

    # sort the temp_vals
    sorted_list = sorted(temp_vals,key = lambda x: x[1])

    if len(sorted_list) > 0:
        val_remove = sorted_list[::-1].pop()
        print(rank_val, val_remove)
        rank_val += 1
        zone_list.remove(val_remove)
    
    return zone_list, rank_val

In [6]:

i = 1

while i < 205:
    
    # handling vertical
    dist_vals = sorted(set([x[0] for x in total_vert]))[::-1]
    for slope in dist_vals:
        total_vert, i = blast_asteroids(slope, total_vert, i)
    print("finished vertical loop")
    # handling zone 1
    dist_vals = sorted(set([x[0] for x in zone_1]))[::-1]
    for slope in dist_vals:
        zone_1, i = blast_asteroids(slope, zone_1, i)
    print("Finished zone 1")
    
    # handling zone 2
    dist_vals = sorted(set([x[0] for x in zone_2]))[::-1]
    for slope in dist_vals:
        zone_2, i = blast_asteroids(slope, zone_2, i)
    print("Finished zone 2")
    
    # handling negative vertical
    dist_vals = sorted(set([x[0] for x in total_neg_vert]))[::-1]
    for slope in dist_vals:
        total_neg_vert, i = blast_asteroids(slope, total_neg_vert, i)

    print("finished negative vert")
    
    # handling zone 3
    dist_vals = sorted(set([x[0] for x in zone_3]))[::-1]
    for slope in dist_vals:
        zone_3, i = blast_asteroids(slope, zone_3, i)
    print("Finished zone 3")
    
    # handling zone 4
    dist_vals = sorted(set([x[0] for x in zone_4]))[::-1]
    for slope in dist_vals:
        zone_4, i = blast_asteroids(slope, zone_4, i)
    print("Finished Zone 4")

1 (None, 1.0, (19, 1))
finished vertical loop
2 (10.0, 10.04987562112089, (20, 10))
3 (8.0, 8.06225774829855, (20, 8))
4 (7.0, 7.0710678118654755, (20, 7))
5 (6.0, 6.082762530298219, (20, 6))
6 (5.0, 5.0990195135927845, (20, 5))
7 (4.5, 9.219544457292887, (21, 9))
8 (4.0, 4.123105625617661, (20, 4))
9 (3.6666666666666665, 11.40175425099138, (22, 11))
10 (3.5, 7.280109889280518, (21, 7))
11 (3.3333333333333335, 10.44030650891055, (22, 10))
12 (3.0, 3.1622776601683795, (20, 3))
13 (2.5, 5.385164807134504, (21, 5))
14 (2.3333333333333335, 7.615773105863909, (22, 7))
15 (2.0, 4.47213595499958, (21, 4))
16 (1.6666666666666667, 5.830951894845301, (22, 5))
17 (1.5, 3.605551275463989, (21, 3))
18 (1.3333333333333333, 5.0, (22, 4))
19 (1.0, 1.4142135623730951, (20, 1))
20 (0.6666666666666666, 3.605551275463989, (22, 2))
21 (0.5, 2.23606797749979, (21, 1))
22 (0.0, 1.0, (20, 0))
Finished zone 1
23 (-0.3333333333333333, 3.1622776601683795, (22, -1))
24 (-0.6666666666666666, 3.605551275463989, (22